In [ ]:
# python 3.11.8
# DLISIO v1.0.1
# numpy v1.26.3
# pandas v2.2.0
# lasio v0.31

from dlisio import lis
import pandas as pd
import os
import lasio
import numpy as np

from IPython.display import display, HTML, Image

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 60%; }
</style>
"""))

In [ ]:
def extract_wellname(f, find_wellname, manualwellname):
    if find_wellname == "Yes":
        records = f.wellsite_data()
        inforec = records[0] # assume well name in is in the first record
        #Can it be structured? Otherwise return an error
        if inforec.isstructured() == True:
            np_array = (inforec.table(simple=True)).T
            df = pd.DataFrame(data=np_array)
            
            wellname_list = []

            items = ['Well Name', 'wellname', 'WN', 'WELL NAME'] ##set all the possible search terms

            for item in items:
                df["string_index"] = df["VALU"].str.find(str(item)).values
                wellname = df.loc[df['string_index'] >= 0, 'VALU'].values.tolist()
                #wellname = wellname[0] #take the first value returned.
                wellname_list.append(wellname)

                df["string_index"] = df["MNEM"].str.find(str(item)).values
                wellname = df.loc[df['string_index'] >= 0, 'VALU'].values.tolist()
                wellname_list.append(wellname)

            wellname_list = [x for x in wellname_list if x != []]
            for wellname in wellname_list:
                wellname = [s.replace('Well Name:', '') for s in wellname]
                wellname = [s.replace(' WN', '') for s in wellname]
                wellname = [s.replace('Well Name ', '') for s in wellname]
                wellname = [s.replace('WN', '') for s in wellname]
                wellname = [s.strip() for s in wellname]
                
            wellname = wellname[0]   ##take the first item in the list
            
            if len(wellname)>0:
                wellname_report_error = ""
            else:
                wellname_report_error = "Could not read header - no name extraction"
        else:
            wellname_report_error = "Could not read header - not structured"
    else:
        wellname = manualwellname
        wellname_report_error = ''
    return wellname, wellname_report_error

In [ ]:
def create_las(curves_df, units, curvenames, wellname, filepath):
    las = lasio.LASFile()
    # write the pandas data to the las file
    las.set_data(curves_df)
    # write the curve metadata from our three lists.
    counter = 0
    for x in curvenames:
        las.curves[x].unit = units[counter]
        counter = counter + 1
    las.well.WELL = wellname
    las.params['LINEAGE'] = lasio.HeaderItem('LINEAGE', value="Python-converted from LIS")
    las.params['ORFILE'] = lasio.HeaderItem('ORFILE', value=filepath)
    return las

In [ ]:
def write_las_file(las, filepath, frame_count, output_folder_location):
    
    filename = os.path.basename(filepath)
    filename = os.path.splitext(filename)[0]
    
    outfile = filename + "_" + "converted_with_python_" + str(frame_count) + ".las"
    outpath = os.path.join(output_folder_location, outfile)

    if not os.path.exists(output_folder_location):
        print("Making output directory: [{}]\n".format(output_folder_location))
        os.makedirs(output_folder_location)

    print("Writing: [{}]\n".format(outpath))
    las.write(outpath, version=2)

In [ ]:
# process each logical file and each sampling rate (frame) at a time. 
# The goal with this function is to only read from the lis file once!
# If you don't care about speed, and need the well names extracted from the lis header, set the find_wellname = Yes
# Otherwise pass in the name of the well into the manualwellname variable.
# The number of logical files times the number of groups of sample rates will determine the number of las files created.
# Curves with no matching frame sample rate will not be included!!  See Fast Channels in the LIS User Guides @dlisio.readthedocs

def lis_to_las(filepath, output_folder_location, manualwellname = "set input", find_wellname = "Yes"):
    frame_count = 0
    with lis.load(filepath) as files:
        for f in files:
            wellname = extract_wellname(f, find_wellname, manualwellname)
            for fs in f.data_format_specs():
                for frame in fs.sample_rates():
                    curve_frames = []
                    #create dataframe
                    units = []
                    curvenames = []
                    meta = lis.curves_metadata(fs, sample_rate=frame)
                    curves = lis.curves(f, fs, sample_rate=frame)
                    curves_df = curves.T
                    names = curves.dtype.names
                    curves_df = pd.DataFrame(data=curves_df, columns=names)
                    
                    #check if dataframe is empty
                    if curves_df.empty:
                        frame_count = frame_count + 1
                        print('Frame number '+ str(frame_count) + " is empty")
                        break
                    
                    #get and set index curve
                    index_curve = fs.index_mnem
                    curves_df = curves_df.set_index(index_curve)
                    
                    #get inventories
                    for n in names:
                        spec = meta[n]
                        units.append(spec.units)
                        curvenames.append(n)
                    
                    # advance the count of sample rate frames
                    frame_count = frame_count + 1
                
                    #Quick quality test
                    unit_len = len(curvenames)
                    name_len = len(units)
                    if unit_len != name_len:
                        warning = "Frame "+ str(frame_count)+" "+"Mismatch in number of names and units; use with caution."
                    else:
                        warning = ""
                    print(warning)
                    
                    las = create_las(curves_df, units, curvenames, wellname, filepath)
                    
                    write_las_file(las, filepath, frame_count, output_folder_location)
                
    report = ("Number of frames = " + str(frame_count))  
    print(report)
        
    return report

In [ ]:
filepath = r"Volve_Well_logs_pr_WELL\15_9-F-4\04.COMPOSITE\WLC_PETROPHYSICAL_COMPOSITE_1.LIS"

In [ ]:
#filepath = r"\Volve_Well_logs_pr_WELL\15_9-F-4\01.MUD_LOG\MUD_LOG_1.LIS"

In [ ]:
output_folder_location = ""

In [ ]:
#results = lis_to_las(filepath, output_folder_location, manualwellname = "Volve", find_wellname="No")

In [ ]:
results = lis_to_las(filepath, output_folder_location, find_wellname="Yes")